![title](../DOCS/img.png)

This image likely serves as a title or banner for the project.

# **What is SOAP ?** #
A SOAP note is a structured method of documentation used by healthcare providers. The acronym SOAP stands for:

* **S - Subjective**: This section captures information reported by the patient, such as their feelings, concerns, and their description of symptoms (e.g., "stomach pain," "nauseated, fatigued"). It includes the patient's chief complaint and the history of their present illness.

* **O - Objective**: This part includes observable, measurable, and factual data collected by the clinician. This encompasses vital signs (temperature, heart rate, blood pressure, etc.), physical exam findings, laboratory results, and imaging data.

* **A - Assessment**: Here, the clinician provides their professional judgment and diagnosis based on the subjective and objective information gathered. It involves an analysis of the patient's condition, potential diagnoses, and the patient's progress.

* **P - Plan**: This section outlines the treatment plan, including any further tests, therapies, medications, referrals to specialists, and follow-up actions.

SOAP notes are a crucial tool for healthcare workers to organize patient information, guide clinical reasoning, and facilitate communication among health professionals. They help ensure consistent and clear documentation, which is essential for quality patient care. This standardized format was developed by Dr. Lawrence Weed in the 1960s.

# What is TxGemma ? #

TxGemma is a collection of open-source machine learning models designed to improve the efficiency of therapeutic development. These models are fine-tuned from Google DeepMind's Gemma 2 architecture using a large dataset (7 million training examples) from the Therapeutics Data Commons (TDC), which includes information on small molecules, proteins, nucleic acids, diseases, and cell lines.

TxGemma models come in various sizes (2B, 9B, and 27B parameters) and are built to Predict therapeutic properties, Perform classification, regression, and generation tasks, Facilitate conversational AI for deeper insights, Support agentic orchestration.

_(Google, "TXGemma: A Family of Lightweight Open Models," Google AI Blog, accessed May 22, 2025, https://blog.google/technology/ai/gemma-open-models/.)_

# What is OMI ? #

OMI dataset consists of 10,000 synthetic dialogues between a patient and clinician, created using the GPT-4 dataset from NoteChat, based on PubMed Central (PMC) case-reports. Accompanying these dialogues are SOAP summaries generated through GPT-4. The dataset is split into 9250 training, 500 validation, and 250 test entries, each containing a dialogue column, a SOAP column, a prompt column, and a ChatML-style conversation format column.

_(Junxian Tang et al., "NoteChat: A Dataset of Synthetic Doctor-Patient Conversations Conditioned on Clinical Notes," arXiv, last modified October 24, 2023, https://arxiv.org/abs/2310.15959.)_

## 1. Setup and Initialization

This section imports necessary libraries and defines file paths for data storage. `pathlib` is used for robust path manipulation, `datasets` for loading data from Hugging Face, `pandas` for data manipulation in DataFrames, and `re` for regular expression operations.

In [1]:
# Import necessary libraries
from pathlib import Path  # For object-oriented path manipulation
from datasets import load_dataset  # For loading datasets from Hugging Face Hub
import pandas as pd  # For data manipulation and analysis using DataFrames
import re  # For regular expression operations, used later for text processing

# Define base data path
DATA_PATH = Path("../data")

# Define specific paths for processed and raw OMI dataset
# OMI_PATH_processed will store the cleaned and transformed data
OMI_PATH_processed = DATA_PATH / "processed" / "omi-health"
# OMI_PATH_raw will store the initially downloaded dataset
OMI_PATH_raw = DATA_PATH / "raw" / "omi-health"

# Create the directory for processed data if it doesn't already exist
# parents=True: creates parent directories if they don't exist
# exist_ok=True: doesn't raise an error if the directory already exists
OMI_PATH_processed.mkdir(parents=True, exist_ok=True)

/opt/miniconda3/envs/SOAP/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 2. Data Loading and Initial Storage

The following cell loads the 'omi-health/medical-dialogue-to-soap-summary' dataset from the Hugging Face Hub. This dataset contains medical dialogues and their corresponding SOAP summaries. After loading, the raw dataset is saved to the disk in the `OMI_PATH_raw` directory for future use and to avoid re-downloading.

In [2]:
# Load the dataset from Hugging Face Hub
# The dataset is identified by "omi-health/medical-dialogue-to-soap-summary"
ds_omi_health = load_dataset("omi-health/medical-dialogue-to-soap-summary")

# Save the loaded dataset to the raw data path defined earlier
# This allows for quicker access later and serves as a backup of the original data format
ds_omi_health.save_to_disk(OMI_PATH_raw)

C:\Users\khal6952\.conda\envs\SOAP\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\khal6952\.cache\huggingface\hub\datasets--omi-health--medical-dialogue-to-soap-summary. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Generating test split: 100%|██████████| 250/250 [00:00<00:00, 20787.76 examples/s]
C:

## 3. Data Conversion and Initial Exploration

The dataset, once loaded, is typically in a Hugging Face `Dataset` object format. For easier manipulation and analysis, especially with tabular data, it's converted into Pandas DataFrames. This section converts the 'train', 'validation', and 'test' splits of the dataset into their respective DataFrames and then displays the first few rows of the training DataFrame (`train_df_omi.head()`) to get a quick overview of its structure and content.

In [3]:
# Convert the 'train' split of the dataset to a Pandas DataFrame if it exists
if 'train' in ds_omi_health:
    train_df_omi = ds_omi_health['train'].to_pandas()

# Convert the 'validation' split to a Pandas DataFrame if it exists
if 'validation' in ds_omi_health:
    val_df_omi = ds_omi_health['validation'].to_pandas()

# Convert the 'test' split to a Pandas DataFrame if it exists
if 'test' in ds_omi_health:
    test_df_omi = ds_omi_health['test'].to_pandas()

# Display the first 5 rows of the training DataFrame to inspect its columns and sample data
train_df_omi.head()

,dialogue,soap,prompt,messages,messages_nosystem
0,"Doctor: Hello, how can I help you today?\nPati...",S: The patient's mother reports that her 13-ye...,Create a Medical SOAP note summary from the di...,"[{'role': 'system', 'content': 'You are an exp...","[{'role': 'user', 'content': 'You are an exper..."
1,"Doctor: Hello, what brings you in today?\nPati...","S: The patient, a 21-month-old male, presented...",Create a Medical SOAP note summary from the di...,"[{'role': 'system', 'content': 'You are an exp...","[{'role': 'user', 'content': 'You are an exper..."
2,"Doctor: Hello, how can I help you today?\nPati...","S: Patient reports experiencing fatigue, night...",Create a Medical SOAP note summary from the di...,"[{'role': 'system', 'content': 'You are an exp...","[{'role': 'user', 'content': 'You are an exper..."
3,"Doctor: Hello, Patient D. How are you feeling ...","S: Patient D, a 60-year-old African American m...",Create a medical SOAP summary of this dialogue.,"[{'role': 'system', 'content': 'You are an exp...","[{'role': 'user', 'content': 'You are an exper..."
4,"Doctor: Hello, I see that you have a history o...","S: The patient, a married woman with a 7-year ...",Create a Medical SOAP note summary from the di...,"[{'role': 'system', 'content': 'You are an exp...","[{'role': 'user', 'content': 'You are an exper..."


### 3.1. DataFrame Information

To get a concise summary of the DataFrame, including the data types of each column and the number of non-null values, the `.info()` method is used. This is helpful for understanding memory usage and identifying columns with missing data.

In [4]:
# Display a summary of the training DataFrame
# This includes the index dtype, column dtypes, non-null values, and memory usage.
train_df_omi.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9250 entries, 0 to 9249
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   dialogue           9250 non-null   object
 1   soap               9250 non-null   object
 2   prompt             9250 non-null   object
 3   messages           9250 non-null   object
 4   messages_nosystem  9250 non-null   object
dtypes: object(5)
memory usage: 361.5+ KB


### 3.2. Inspecting Individual Data Entries

Let's look at a single example from the training data to better understand the content of the 'dialogue' and 'soap' columns. We use `.iloc[0]` to access the first row.

In [5]:
# Let's check one example: display the 'dialogue' content of the first entry (index 0)
train_df_omi.iloc[0]['dialogue']

"Doctor: Hello, how can I help you today?\nPatient: My son has been having some issues with speech and development. He's 13 years old now.\nDoctor: I see. Can you tell me more about his symptoms? Does he have any issues with muscle tone or hypotonia?\nPatient: No, he doesn't have hypotonia. But he has mild to moderate speech and developmental delay, and he's been diagnosed with attention deficit disorder.\nDoctor: Thank you for sharing that information. We'll run some tests, including an MRI, to get a better understanding of your son's condition. \n(After the tests)\nDoctor: The MRI results are in, and I'm glad to say that there are no structural brain anomalies. However, I did notice some physical characteristics. Does your son have any facial features like retrognathia, mild hypertelorism, or a slightly elongated philtrum and thin upper lip?\nPatient: Yes, he has all of those features. His hands are also broad and short. And his feet have mild syndactyly of the second and third toe, 

In [6]:
# Display the 'soap' note content of the first entry (index 0)
train_df_omi.iloc[0]['soap']

"S: The patient's mother reports that her 13-year-old son has mild to moderate speech and developmental delays and has been diagnosed with attention deficit disorder. She denies any issues with muscle tone or hypotonia. The patient also exhibits certain physical characteristics, including retrognathia, mild hypertelorism, an elongated philtrum, thin upper lip, broad and short hands, mild syndactyly of the second and third toes, and a sandal gap in both feet.\nO: An MRI of the brain showed no structural anomalies. Whole Exome Sequencing (WES) revealed a de novo frameshift variant Chr1(GRCh37):g.244217335del, NM_205768.2(ZBTB18):c.259del(p.(Leu87Cysfs*21)), indicating a premature termination codon located more than 400 codons upstream of the canonical termination codon.\nA: The primary diagnosis is a genetic disorder associated with the identified frameshift mutation, which likely contributes to the patient's speech and developmental delays and attention deficit disorder. The physical ch

## 4. Feature Engineering: Extracting Event Tags from Dialogues

Some dialogues may contain tags indicating events or time progression, such as `(After the tests)` or `[After 3 weeks of therapy]`. These tags can provide contextual information. The following function `extract_dialogue_tags` uses regular expressions to find and extract such tags if they appear on their own lines within the dialogue text.

In [7]:
# Define a function to extract event tags from dialogue text
def extract_dialogue_tags(dialogue):
    # Check if the dialogue is NaN (Not a Number), which can occur for missing values
    if pd.isna(dialogue):
        return []  # Return an empty list if dialogue is missing

    # Regular expression to find text enclosed in parentheses () or square brackets []
    # that appears on its own line (or effectively on its own line due to surrounding whitespace and newline).
    # Details of the regex:
    # \n\s*: Matches a newline character followed by zero or more whitespace characters (start of line with potential indent).
    # (\(.*?\)|\[.*?\]): Capturing group for the tag itself.
    #   \(.*?\): Matches anything inside literal parentheses (non-greedy, i.e., shortest match).
    #   |: OR operator.
    #   \[.*?\]: Matches anything inside literal square brackets (non-greedy).
    # \s*: Matches zero or more whitespace characters after the tag on the same line.
    # (?:\n|$): Non-capturing group that matches either a newline character or the end of the string.
    #            This ensures the tag is effectively on a line by itself or at the end of the dialogue.
    pattern = r"\n\s*(\(.*?\)|\[.*?\])\s*(?:\n|$)"
    
    # Find all occurrences of the pattern in the dialogue (converted to string to be safe)
    tags = re.findall(pattern, str(dialogue))
    
    # Convert the list of found tags to a string representation of the list (e.g., "['(tag1)', '(tag2)']")
    text = str(tags)
    return text

### 4.1. Testing the Tag Extraction Function

Let's test the `extract_dialogue_tags` function on the first few entries of the training data to see if it correctly identifies and extracts the tags.

In [8]:
# Iterate over the first 5 rows of the training DataFrame
for index, row in train_df_omi.head().iterrows():
    # Extract tags from the 'dialogue' column of the current row
    tags_on_new_lines = extract_dialogue_tags(row['dialogue'])
    # Print the extracted tags for each dialogue
    print(f"\nTags found on new lines: {tags_on_new_lines}")


Tags found on new lines: ['(After the tests)']

Tags found on new lines: ['[After the tests]', '[After 3 weeks of therapy]']

Tags found on new lines: []

Tags found on new lines: []

Tags found on new lines: []


### 4.2. Applying Tag Extraction to the DataFrame

Now, apply the `extract_dialogue_tags` function to the entire 'dialogue' column of the training DataFrame. The results, which are series of lists (or rather, string representations of lists), are then concatenated as new columns to the original DataFrame. The primary new column containing these tags is then renamed to 'event_tags'.

In [9]:
# Apply the tag extraction function to the 'dialogue' column of the training DataFrame.
# .apply(pd.Series) is used to convert the list-like results from extract_dialogue_tags 
# (which returns a string representation of a list) into separate columns if the string represented a list of multiple items.
# However, since extract_dialogue_tags returns a single string, this will result in one new column (column 0).
tags_df = train_df_omi['dialogue'].apply(extract_dialogue_tags).apply(pd.Series)

# Concatenate the new tags DataFrame (tags_df) with the original training DataFrame (train_df_omi)
# axis=1 means concatenate column-wise
train_df_omi = pd.concat([train_df_omi, tags_df], axis=1)

# Rename the newly added column (which is initially named 0) to 'event_tags'
train_df_omi = train_df_omi.rename(columns={
    0: 'event_tags'
})

### 4.3. Displaying DataFrame with Extracted Tags

Let's view the DataFrame again to see the new 'event_tags' column.

In [10]:
# Display the entire training DataFrame to show the newly added 'event_tags' column
train_df_omi

,dialogue,soap,prompt,messages,messages_nosystem,event_tags
0,"Doctor: Hello, how can I help you today?\nPati...",S: The patient's mother reports that her 13-ye...,Create a Medical SOAP note summary from the di...,"[{'role': 'system', 'content': 'You are an exp...","[{'role': 'user', 'content': 'You are an exper...",['(After the tests)']
1,"Doctor: Hello, what brings you in today?\nPati...","S: The patient, a 21-month-old male, presented...",Create a Medical SOAP note summary from the di...,"[{'role': 'system', 'content': 'You are an exp...","[{'role': 'user', 'content': 'You are an exper...","['[After the tests]', '[After 3 weeks of thera..."
2,"Doctor: Hello, how can I help you today?\nPati...","S: Patient reports experiencing fatigue, night...",Create a Medical SOAP note summary from the di...,"[{'role': 'system', 'content': 'You are an exp...","[{'role': 'user', 'content': 'You are an exper...",[]
3,"Doctor: Hello, Patient D. How are you feeling ...","S: Patient D, a 60-year-old African American m...",Create a medical SOAP summary of this dialogue.,"[{'role': 'system', 'content': 'You are an exp...","[{'role': 'user', 'content': 'You are an exper...",[]
4,"Doctor: Hello, I see that you have a history o...","S: The patient, a married woman with a 7-year ...",Create a Medical SOAP note summary from the di...,"[{'role': 'system', 'content': 'You are an exp...","[{'role': 'user', 'content': 'You are an exper...",[]
...,...,...,...,...,...,...
9245,"Doctor: Hello, I see you're here for a problem...",S: The patient reports difficulty seeing in th...,Create a Medical SOAP note summary from the di...,"[{'role': 'system', 'content': 'You are an exp...","[{'role': 'user', 'content': 'You are an exper...",[]
9246,Doctor: Hi there! I see you have brought your ...,"S: The patient, a 3-year-old neutered male Box...",Create a Medical SOAP note summary from the di...,"[{'role': 'system', 'content': 'You are an exp...","[{'role': 'user', 'content': 'You are an exper...",[]
9247,"Doctor: Hello there, how can I help you today?...",S: The patient is a 29-year-old obese male wit...,Create a Medical SOAP note summary from the di...,"[{'role': 'system', 'content': 'You are an exp...","[{'role': 'user', 'content': 'You are an exper...",[]
9248,"Doctor: Hello, I understand that you've been t...",S: The patient reports feeling weak but managi...,Create a medical SOAP summary of this dialogue.,"[{'role': 'system', 'content': 'You are an exp...","[{'role': 'user', 'content': 'You are an exper...",[]


## 5. Saving Intermediate Processed Data (Version 1)

At this stage, the DataFrame includes the original data plus the extracted 'event_tags'. This version of the processed data is saved to a CSV file named `train_v1.csv` in the `OMI_PATH_processed` directory. This serves as a checkpoint.

In [11]:
# Save the current state of the training DataFrame to a CSV file
# index=False prevents Pandas from writing the DataFrame index as a column in the CSV
train_df_omi.to_csv(OMI_PATH_processed / 'train_v1.csv', index=False)

## 6. Further Processing: Splitting SOAP Notes into Components

The 'soap' column contains the full SOAP note as a single string, with sections (Subjective, Objective, Assessment, Plan) typically separated by newlines and prefixed with 'S:', 'O:', 'A:', 'P:'. The `split_soap` function is designed to parse this string and separate these components into individual columns. This makes each part of the SOAP note directly accessible for targeted analysis or model training.

In [12]:
# Define a function to split the 'soap' string into S, O, A, P components
def split_soap(soap):
    # Split the SOAP note string by newline characters to get individual lines/components
    components = soap.split('\n')
    soap_dict = {} # Initialize an empty dictionary to store the components
    # Iterate through each component line
    for component in components:
        if ':' in component: # Ensure there's a colon to split by (e.g., "S: ...")
            # Split the component line by the first colon
            key, value = component.split(':', 1)
            # Store the stripped key (e.g., 'S') and stripped value in the dictionary
            soap_dict[key.strip()] = value.strip()
    return soap_dict

# Apply the split_soap function to the 'soap' column of the training DataFrame.
# .apply(pd.Series) converts the dictionary returned by split_soap for each row into new columns.
soap_df = train_df_omi['soap'].apply(split_soap).apply(pd.Series)

# Concatenate the original training DataFrame with the new DataFrame containing separate S, O, A, P columns
train_df_omi = pd.concat([train_df_omi, soap_df], axis=1)

# Optional: Drop the original 'soap' column as its content is now split into new columns
# train_df_omi = train_df_omi.drop(columns=['soap'])

# Rename the new columns from 'S', 'O', 'A', 'P' to more descriptive names
train_df_omi = train_df_omi.rename(columns={
    'S': 'subjective',
    'O': 'objective',
    'A': 'assessment',
    'P': 'plan'
})

# Display the first 5 rows of the updated DataFrame to see the new SOAP component columns
train_df_omi.head()

,dialogue,soap,prompt,messages,messages_nosystem,event_tags,subjective,objective,assessment,plan,...,- Slit lamp examination of the right eye,- Scheimpflug densitometry,- Ears,- Nose,- Oral,- Neck,- Additional findings on the following day,- Laboratory results,- Transvaginal ultrasound showing diffuse fibromatosis with two uterine masses,- Abdominal CT indicated increased uterine volume with two masses
0,"Doctor: Hello, how can I help you today?\nPati...",S: The patient's mother reports that her 13-ye...,Create a Medical SOAP note summary from the di...,"[{'role': 'system', 'content': 'You are an exp...","[{'role': 'user', 'content': 'You are an exper...",['(After the tests)'],The patient's mother reports that her 13-year-...,An MRI of the brain showed no structural anoma...,The primary diagnosis is a genetic disorder as...,The management plan includes regular follow-up...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"Doctor: Hello, what brings you in today?\nPati...","S: The patient, a 21-month-old male, presented...",Create a Medical SOAP note summary from the di...,"[{'role': 'system', 'content': 'You are an exp...","[{'role': 'user', 'content': 'You are an exper...","['[After the tests]', '[After 3 weeks of thera...","The patient, a 21-month-old male, presented wi...",Hip ultrasound showed no joint effusion. Spine...,Primary diagnosis is Spondylodiscitis with ass...,Initiated broad-spectrum intravenous therapy w...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"Doctor: Hello, how can I help you today?\nPati...","S: Patient reports experiencing fatigue, night...",Create a Medical SOAP note summary from the di...,"[{'role': 'system', 'content': 'You are an exp...","[{'role': 'user', 'content': 'You are an exper...",[],"Patient reports experiencing fatigue, night sw...","Vital signs normal. BMI 37.2 kg/m2, weight 263...",The patient presents with symptoms suggestive ...,Continue current medications. Schedule follow-...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"Doctor: Hello, Patient D. How are you feeling ...","S: Patient D, a 60-year-old African American m...",Create a medical SOAP summary of this dialogue.,"[{'role': 'system', 'content': 'You are an exp...","[{'role': 'user', 'content': 'You are an exper...",[],"Patient D, a 60-year-old African American male...",Patient is currently asymptomatic. No physical...,Patient D is at an increased risk for prostate...,Plan to have a detailed conversation about PSA...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"Doctor: Hello, I see that you have a history o...","S: The patient, a married woman with a 7-year ...",Create a Medical SOAP note summary from the di...,"[{'role': 'system', 'content': 'You are an exp...","[{'role': 'user', 'content': 'You are an exper...",[],"The patient, a married woman with a 7-year his...",Physical examination confirmed hirsutism and m...,The primary diagnosis is Polycystic Ovarian Sy...,The management plan includes proceeding with i...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## 7. Final Data Selection and Saving (Version 2)

After splitting the SOAP notes, the DataFrame contains many columns. For the final processed dataset, we select only the essential columns: the original 'dialogue' and the newly created 'subjective', 'objective', 'assessment', and 'plan' columns. This refined DataFrame is then saved as `train_v2.csv`. This version is likely the one intended for model training or further specific analysis where individual SOAP components are needed.

In [13]:
# Select only the necessary columns for the final training dataset version
# These are the dialogue and the individual components of the SOAP note.
train_df_omi = train_df_omi[['dialogue', 'subjective', 'objective', 'assessment', 'plan']]

# Save this final version of the training DataFrame to a CSV file
train_df_omi.to_csv(OMI_PATH_processed / 'train_v2.csv', index=False)

## 8. Data Analysis: Finding the Longest Dialogue

As a part of understanding the dataset, it's often useful to find characteristics like the length of text inputs. The `find_longest_dialogue` function is defined to identify the dialogue with the maximum character length in a given DataFrame. This can be helpful for setting maximum token limits for language models or identifying potential outliers.

In [14]:
# Define a function to find the longest dialogue in a DataFrame
def find_longest_dialogue(df, dialogue_column='dialogue'):
    """
    Finds the longest dialogue in a DataFrame based on character length.

    Args:
        df (pd.DataFrame): The DataFrame containing the dialogue data.
        dialogue_column (str, optional): The name of the column containing the dialogues.
                                         Defaults to 'dialogue'.

    Returns:
        tuple: A tuple containing:
               - int: The index of the longest dialogue.
               - int: The length (number of characters) of the longest dialogue.
    """

    # Calculate the length (number of characters) of each dialogue in the specified column
    dialogue_lengths = df[dialogue_column].apply(len)

    # Find the index of the dialogue with the maximum length
    longest_dialogue_index = dialogue_lengths.idxmax()

    # Get the actual text of the longest dialogue (though we only return its length here as per function design)
    # longest_dialogue_text_content = df.loc[longest_dialogue_index, dialogue_column]

    # Return the index and the length of the longest dialogue
    return longest_dialogue_index, len(df.loc[longest_dialogue_index, dialogue_column]) # Corrected to return length

### 8.1. Applying the Longest Dialogue Finder

The `find_longest_dialogue` function is now applied to the `train_df_omi` (which at this point contains the columns from `train_v2.csv`). The index and length of the longest dialogue are printed. This gives an idea of the maximum input size the model might encounter from this dataset.

In [15]:
# Apply the function to find the longest dialogue in the training DataFrame (train_df_omi)
# The comment indicates it could also be used on val_df_omi or test_df_omi if they were processed similarly.
longest_index, longest_dialogue_length = find_longest_dialogue(train_df_omi)

# Print the index of the longest dialogue
print(f"The longest dialogue is at index: {longest_index}\n")
# Print the length (number of characters) of the longest dialogue text
print("Length of the longest dialogue text (characters):\n", longest_dialogue_length)

The longest dialogue is at index: 1823

Longest dialogue text:
 3730
